#### NLP | MVP

# Coronavirus Tweets: Spring 2020<a id='top'></a> 

# 

## **Analysis Goal**  
The client, [name](link), ...
wants to ... and 
requested a model to find a pattern ... in tweets in April. 

Impetus?

**RQ:** ....?

## **Process**
**Data source:** 
Coronavirus COVID-19 Tweets [early](https://www.kaggle.com/datasets/smid80/coronavirus-covid19-tweets-early-april) and [late](https://www.kaggle.com/datasets/smid80/coronavirus-covid19-tweets-late-april) April


**Models:** 
* [name] model, fit a [model name](#??):
    * eval metric
* [name] model, fit a [model name](#??):
    * eval metric


## **Preliminary Visualizations**
[Figure 1. Name](#??)<br/>
[Figure 1. Name](#??)<br/>

## **Preliminary Conclusions**
The [name] model ... showed....<br/>

The next step .... 

In [2]:
import pandas as pd
import glob 
pd.set_option('display.max_colwidth', None)


## 1 | Dataset: Coronavirus Tweets ([early](https://www.kaggle.com/datasets/smid80/coronavirus-covid19-tweets-early-april) and [late](https://www.kaggle.com/datasets/smid80/coronavirus-covid19-tweets-late-april) April) <a id='1'></a>  

In [ ]:
# import one csv file to see data
single_df = pd.read_csv('20200401.csv')

In [ ]:
single_df.head(2)

In [ ]:
single_df.info()

In [ ]:
# keep only the following 7 columns on full data import 

 2   created_at            591480 non-null  object 
 3   screen_name           591480 non-null  object 
 4   text                  591480 non-null  object 
 13  country_code          26040 non-null   object 
 18  account_lang          0 non-null       float64
 20  verified              591480 non-null  bool   
 21  lang                  591480 non-null  object 

In [ ]:
# read list of April 2020 files, while ommitting columns above

path = r'/Users/sandraparedes/Documents/GitHub/metis_dsml/05_nlp/g00-nlp-project/data' 
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0, usecols=[2,3,4,13,18,20,21])
    li.append(df)
df = pd.concat(li, axis=0, ignore_index=True)


In [ ]:
df.head()

In [ ]:
df.info()

## 2 | Exploratory Data Analysis<a id='2'></a>  

In [ ]:
df.info()

In [ ]:
df['country_code'].unique()

In [ ]:
df['lang'].unique()

In [ ]:
print('English entries:', (df[df["lang"] == 'en'].count())['lang'])

In [ ]:
print('US entries:', (df[df["country_code"] == 'US'].count())['country_code'])

In [ ]:
eng_us_df = df[(df['country_code'] == 'US') &
               (df['lang'] == 'en')]
eng_us_df.info()


### English + US df: 138,789 entries

In [ ]:
eng_us_df.head(10)

In [ ]:
eng_us_df['text'].head(10)

In [5]:
# #save cleaned df as tweet_df

# tweet_df = eng_us_df 
# tweet_df.to_pickle('coronavirus_tweets_df.pkl')
# tweet_df.to_csv(r'//Users/sandraparedes/Documents/GitHub/metis_dsml/05_nlp/g00-nlp-project/data/coronavirus_tweets_df.csv', index=False)


In [6]:
#read df in
df = pd.read_csv('coronavirus_tweets_df.csv', low_memory=False)

### 2a | [start here with nlp](http://localhost:8888/notebooks/Documents/GitHub/metis_dsml/05_nlp/g1-nlp-overview/NLP_Supervised_Learning.ipynb)<a id='2a'></a>  

### 3 | [sentiment analysis](http://localhost:8888/notebooks/Documents/GitHub/metis_dsml/05_nlp/g1-nlp-overview/nlp_unsupervised_nlp_exercises7.ipynb) code blocks<a id='3'></a>  

### 4 | [dim redux](http://localhost:8888/notebooks/Documents/GitHub/metis_dsml/05_nlp/g2-dimensionality-reduction/nlp_unsupervised_dimensionality_reduction_exercises3.ipynb) and [PCA](http://localhost:8888/notebooks/Documents/GitHub/metis_dsml/05_nlp/g2-dimensionality-reduction/PCA_digits.ipynb)<a id='4'></a>  

### 4 | [LSA + NMF](http://localhost:8888/notebooks/Documents/GitHub/metis_dsml/05_nlp/g3-topic-modeling/Topic_Modeling_LSA_NMF.ipynb) [LSA chapter ex](http://localhost:8888/notebooks/Documents/GitHub/metis_dsml/05_nlp/g3-topic-modeling/nlp_unsupervised_topic_modeling_exercises2.ipynb) 